In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split


In [5]:
base_dir = "D:\\New folder\\archive (2)\\catvsdog\\PetImages"
cat_dir = os.path.join(base_dir, 'Cat')
dog_dir = os.path.join(base_dir, 'Dog')


In [6]:
# Directories for train, test, validation splits
for split in ['train', 'test', 'validation']:
    for category in ['Cat', 'Dog']:
        os.makedirs(os.path.join(base_dir, split, category), exist_ok=True)


In [7]:
cat_files = [os.path.join(cat_dir, file) for file in os.listdir(cat_dir)]
dog_files = [os.path.join(dog_dir, file) for file in os.listdir(dog_dir)]


In [8]:
train_cat, temp_cat = train_test_split(cat_files, test_size=0.4, random_state=42)
test_cat, val_cat = train_test_split(temp_cat, test_size=0.5, random_state=42)

train_dog, temp_dog = train_test_split(dog_files, test_size=0.4, random_state=42)
test_dog, val_dog = train_test_split(temp_dog, test_size=0.5, random_state=42)


In [9]:
def copy_files(file_list, split, category):
    for file in file_list:
        dest = os.path.join(base_dir, split, category, os.path.basename(file))
        shutil.copyfile(file, dest)


copy_files(train_cat, 'train', 'cat')
copy_files(test_cat, 'test', 'cat')
copy_files(val_cat, 'validation', 'cat')


copy_files(train_dog, 'train', 'dog')
copy_files(test_dog, 'test', 'dog')
copy_files(val_dog, 'validation', 'dog')


In [11]:
import os
from PIL import Image

def remove_corrupted_images(directory):
    num_corrupted = 0
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            img = Image.open(file_path)
            img.verify()  
        except (IOError, SyntaxError) as e:
            print(f"Removing corrupted file: {file_path}")
            os.remove(file_path)
            num_corrupted += 1
    print(f"Removed {num_corrupted} corrupted images from {directory}")

base_dir = "D:\\New folder\\archive (2)\\catvsdog\\PetImages"
categories = ['train/cat', 'train/dog', 'validation/cat', 'validation/dog', 'test/cat', 'test/dog']

for category in categories:
    directory = os.path.join(base_dir, category)
    remove_corrupted_images(directory)


Removed 0 corrupted images from D:\New folder\archive (2)\catvsdog\PetImages\train/cat
Removing corrupted file: D:\New folder\archive (2)\catvsdog\PetImages\train/dog\11702.jpg
Removed 1 corrupted images from D:\New folder\archive (2)\catvsdog\PetImages\train/dog
Removed 0 corrupted images from D:\New folder\archive (2)\catvsdog\PetImages\validation/cat
Removed 0 corrupted images from D:\New folder\archive (2)\catvsdog\PetImages\validation/dog
Removing corrupted file: D:\New folder\archive (2)\catvsdog\PetImages\test/cat\666.jpg
Removing corrupted file: D:\New folder\archive (2)\catvsdog\PetImages\test/cat\Thumbs.db
Removed 2 corrupted images from D:\New folder\archive (2)\catvsdog\PetImages\test/cat
Removing corrupted file: D:\New folder\archive (2)\catvsdog\PetImages\test/dog\Thumbs.db
Removed 1 corrupted images from D:\New folder\archive (2)\catvsdog\PetImages\test/dog


In [6]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


base_dir = "D:\\New folder\\archive (2)\\catvsdog\\PetImages"
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)




Found 14999 images belonging to 2 classes.
Found 5002 images belonging to 2 classes.
Found 4997 images belonging to 2 classes.


In [7]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


base_dir = "D:\\New folder\\archive (2)\\catvsdog\\PetImages"
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)




Found 14999 images belonging to 2 classes.
Found 5002 images belonging to 2 classes.
Found 4997 images belonging to 2 classes.


In [8]:
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import Xception

conv_base = Xception(
    weights='imagenet',
    include_top=False,
    input_shape=(150, 150, 3)
)



conv_base.trainable = True

set_trainable = False


for layer in conv_base.layers:
    if "block14" in layer.name:
        set_trainable = True
    if set_trainable:
        layer.trainable = True

    else:
        layer.trainable = False


for layer in conv_base.layers:
    print(layer.name, layer.trainable)
    



input_layer_2 False
block1_conv1 False
block1_conv1_bn False
block1_conv1_act False
block1_conv2 False
block1_conv2_bn False
block1_conv2_act False
block2_sepconv1 False
block2_sepconv1_bn False
block2_sepconv2_act False
block2_sepconv2 False
block2_sepconv2_bn False
conv2d_4 False
block2_pool False
batch_normalization_4 False
add_12 False
block3_sepconv1_act False
block3_sepconv1 False
block3_sepconv1_bn False
block3_sepconv2_act False
block3_sepconv2 False
block3_sepconv2_bn False
conv2d_5 False
block3_pool False
batch_normalization_5 False
add_13 False
block4_sepconv1_act False
block4_sepconv1 False
block4_sepconv1_bn False
block4_sepconv2_act False
block4_sepconv2 False
block4_sepconv2_bn False
conv2d_6 False
block4_pool False
batch_normalization_6 False
add_14 False
block5_sepconv1_act False
block5_sepconv1 False
block5_sepconv1_bn False
block5_sepconv2_act False
block5_sepconv2 False
block5_sepconv2_bn False
block5_sepconv3_act False
block5_sepconv3 False
block5_sepconv3_bn False

In [9]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dense(128,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

model.compile(optimizer="adam",loss='binary_crossentropy',metrics=["accuracy"])

In [10]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ xception (Functional)                │ (None, 5, 5, 2048)          │      20,861,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 51200)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 256)                 │      13,107,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 34,001,961 (129.71 MB)

 Trainable params: 17,889,281 (68.24 MB)

 Non-trainable params: 16,112,680 (61.46 MB)

In [5]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50
)


Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 75s 748ms/step - accuracy: 0.9331 - loss: 0.1758 - val_accuracy: 0.9630 - val_loss: 0.1163
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 74s 738ms/step - accuracy: 0.9383 - loss: 0.1804 - val_accuracy: 0.9720 - val_loss: 0.0840
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 74s 741ms/step - accuracy: 0.9435 - loss: 0.1468 - val_accuracy: 0.9720 - val_loss: 0.0638
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 74s 746ms/step - accuracy: 0.9477 - loss: 0.1344 - val_accuracy: 0.9660 - val_loss: 0.0807
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 79s 788ms/step - accuracy: 0.9288 - loss: 0.1546 - val_accuracy: 0.9700 - val_loss: 0.0766
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 59s 589ms/step - accuracy: 0.9374 - loss: 0.1571 - val_accuracy: 1.0000 - val_loss: 2.1619e-06
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 75s 750ms/step - accuracy: 0.9535 - loss: 0.1193 - val_accuracy: 0.9690 - val_loss: 0.0902
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 46s 459ms/step - accuracy: 0.9606 - los